In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import numpy as np  # Add this import statement for NumPy
from tensorflow import keras

# Load the Keras model
model_path = '/content/drive/MyDrive/CAPSTONE/ML/Model/95%/latest/ikan_model_latest.h5'
model = keras.models.load_model(model_path)

# Convert the model to TFLite format with metadata
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Define normalization parameters
mean = [0.485, 0.456, 0.406]  # Example mean values for normalization
std = [0.229, 0.224, 0.225]  # Example standard deviation values for normalization

# Set normalization options in the converter
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS  # enable TensorFlow ops.
]
converter.inference_input_type = tf.uint8  # Assuming your model expects uint8 input
converter.inference_output_type = tf.float32  # Assuming your model outputs float32

# Prepare metadata
class_indices = {'Bawal Putih': 0, 'Nila': 1, 'Pari': 2, 'Tongkol': 3, 'Tuna': 4}
labels = list(class_indices.keys())

# Add metadata
model_meta = tf.saved_model.SaveOptions(namespace_whitelist=labels)

# Define a function to generate representative data
def representative_dataset_gen():
    # Generate a sample input (modify this according to your input shape and data)
    input_shape = (1, 224, 224, 3)
    for _ in range(num_calibration_steps):
        yield [np.random.rand(*input_shape).astype(np.float32)]

# Set the number of calibration steps
num_calibration_steps = 100

# Set the representative dataset
converter.representative_dataset = representative_dataset_gen

# Convert the model
tflite_model = converter.convert()

# Save the TFLite model
tflite_model_path = '/content/drive/MyDrive/CAPSTONE/ML/Model/95%/latest/ikan_model_latest.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

print(f'TFLite model saved to {tflite_model_path}')


/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


TFLite model saved to /content/drive/MyDrive/CAPSTONE/ML/Model/95%/latest/ikan_model_latest.tflite


In [ ]:
# Load your TFLite model
tflite_model_path = '/content/drive/MyDrive/CAPSTONE/ML/Model/95%/ikan_model_v1.tflite'
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# Get input tensor details
input_details = interpreter.get_input_details()
input_dtype = input_details[0]['dtype']
print("Input tensor data type:", input_dtype)

# Get output tensor details
output_details = interpreter.get_output_details()
output_dtype = output_details[0]['dtype']
print("Output tensor data type:", output_dtype)

Input tensor data type: <class 'numpy.uint8'>
Output tensor data type: <class 'numpy.float32'>
